# Trabajo Final Almacenamiento y Captura de Datos

Integrantes:
- Diego León
- Pablo Madariaga

In [23]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
import time  
import time  
import requests
import re

## Web scrapping

### inputs

In [6]:
API_KEY = "insertar_token_de_google"
URL = 'https://www.portalinmobiliario.com/'
URL_UF = 'https://www.valoruf.cl/'
TIPO_CONTRATO = 'arriendo' # venta, arriendo o arriendo_temporal
TIPO_INMUEBLE = 'departamentos' # dpto, casa u oficina.
UBICACION_INMUEBLE = 'las condes' # la comuna de la búsqueda
MONTO_MINIMO = 500000 # monto mínimo de la búsqueda
MONTO_MAXIMO = 1300000 # monto máximo de la búsqueda
CANT_PAGINAS = 2 # número de páginas a recorrer
RADIO_BUSQUEDA = '300' # radio (en metros) de búsqueda de lugares cercanos
BUSQUEDA_RUBROS = ['restaurante', 'supermercado'] # rubro de lugares cercanos

### 1a página: búsqueda inicial

La única búsqueda en chat GPT fue para identificar dinámicamente la n-esima opción asociada
a la búsqueda de la comuna

In [7]:
tipo_contrato_dict = {
    "venta": ":R2l5r:-menu-list-option-242075",
    "arriendo": ":R2l5r:-menu-list-option-242073",
    "arriendo temporal": ":R2l5r:-menu-list-option-242074"
}

tipo_inmueble_dict = {
    "departamentos": ":R4l5r:-menu-list-option-MLC1472_242062",
    "casas": ":R4l5r:-menu-list-option-MLC1466_242060",
    "oficinas": ":R4l5r:-menu-list-option-MLC1478_242067",
    "parcelas": ":R4l5r:-menu-list-option-MLC1496_242070",
    "locales": ":R4l5r:-menu-list-option-MLC50610_242065",
    "terrenos": ":R4l5r:-menu-list-option-MLC152992_245004",
    "sitios": ":R4l5r:-menu-list-option-MLC50613_245008",
    "bodegas": ":R4l5r:-menu-list-option-MLC50564_245003",
    "industriales": ":R4l5r:-menu-list-option-MLC50617_245009",
    "agricolas": ":R4l5r:-menu-list-option-MLC50623_242059",
    "otros inmuebles": ":R4l5r:-menu-list-option-MLC1892_242068",
    "estacionamientos": ":R4l5r:-menu-list-option-MLC50620_242064",
    "loteos": ":R4l5r:-menu-list-option-MLC1493_245010",
}

In [8]:
# Configuramos el driver para el navegador Edge
driver = webdriver.Edge()
driver.set_window_size(1200, 800)  # Ancho: 1200px, Alto: 800px

driver.get(URL)

# Establecemos una espera implícita de 5 segundos para que la página cargue completamente
driver.implicitly_wait(10)

buscador_comuna = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.ID, ":Rml5r:"))
)

# Ingresa una comuna o ciudad
buscador_comuna.send_keys(UBICACION_INMUEBLE)

ul_comunas = "andes-list.faceted-search-desktop-searchbox__list.andes-list--default.andes-list--selectable"
resultados = WebDriverWait(driver, 20).until(
    EC.presence_of_element_located((By.CLASS_NAME, ul_comunas))
)
# seleccionar primera opcion de comuna
primer_boton = resultados.find_element(By.XPATH, "./li[1]/button")
primer_boton.click()


# desplegar la lista de tipo de contrato y clickear el tipo deseado
lista_tipo_contrato = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.ID, ":R2l5r:-trigger"))
)
lista_tipo_contrato.click()
select_tipo_contrato = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.ID, tipo_contrato_dict[TIPO_CONTRATO]))
)
select_tipo_contrato.click()

tipo_inmueble = TIPO_INMUEBLE.lower().replace('á', 'a').replace('é', 'e').\
    replace('í', 'i').replace('ó', 'o').replace('ú', 'u')
lista_tipo_inmueble = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.ID, ":R4l5r:-trigger"))
)
lista_tipo_inmueble.click()

select_tipo_inmueble = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.ID, tipo_inmueble_dict[tipo_inmueble]))
)
select_tipo_inmueble.click()


# apretar buscar
buscador = driver.find_element('id', ':R355r:')
buscador.click()


### 2do página: selección de restricciones

seleccionar rango de precio 

In [9]:
ingreso_precio_min = driver.find_element('id', ':R5chjmee:')
ingreso_precio_min.send_keys(MONTO_MINIMO)

ingreso_precio_max = driver.find_element('id', ':R6chjmee:')
ingreso_precio_max.send_keys(MONTO_MAXIMO)
ingreso_precio_max.send_keys(Keys.RETURN)

recorrer inmuebles

- Para recorrer las páginas se decide navegar entre las urls implícitas en el botón siguiente.
Esto porque al intentar clickear el botón a veces se bloquea y a veces no, lo cual es un comportamiento
intermitente no deseable

- Se agrega una columna temporal "Tipo Moneda" para saber que propiedades tienen el precio en UF

In [10]:
from selenium.common.exceptions import InvalidArgumentException

titulos, precios, direcciones, links_props = [], [], [], []
for i in range(CANT_PAGINAS):

    # Extraer todos los títulos de las propiedades
    titulos_values = driver.find_elements(By.XPATH, "//h2[@class='poly-box poly-component__title']/a")
    titulos += [t.text for t in titulos_values]

    # Extraer todos los precios de las propiedades
    precios_values = driver.find_elements(By.XPATH, "//div[@class='poly-component__price']//span[@class='andes-money-amount__fraction']")
    precios += [p.text for p in precios_values]

    # Intentar con un XPath más general para encontrar la dirección
    direcciones_values = driver.find_elements(By.XPATH, "//span[contains(@class, 'poly-component__location')]")
    direcciones += [d.text for d in direcciones_values]

    # Lo mismo para los enlaces
    links_props_values = driver.find_elements(By.XPATH,  "//h2[@class='poly-box poly-component__title']/a")
    links_props += [l.get_attribute('href') for l in links_props_values]

    if i<CANT_PAGINAS-1: # hacer click en siguiente
        try:
            boton_siguiente = driver.find_element(By.XPATH, "//li[contains(@class, 'andes-pagination__button--next')]/a")
            url_siguiente = boton_siguiente.get_attribute("href")
            driver.get(url_siguiente)
        except InvalidArgumentException: # se piden mas paginas de las que hay
            pass
data_scrapping = pd.DataFrame({
    "Título": titulos,
    "Precio": precios,
    "Dirección": direcciones,
    "Enlace": links_props
})

# diferenciar precios en UF
data_scrapping['Precio'] = data_scrapping['Precio'].str.replace('.', '')
data_scrapping['Precio'] = data_scrapping['Precio'].astype(int)
data_scrapping['Tipo moneda'] = 'CLP'
out_borders = (data_scrapping['Precio']>MONTO_MAXIMO) | (data_scrapping['Precio']<MONTO_MINIMO)
data_scrapping.loc[out_borders, 'Tipo moneda'] = 'UF'
data_scrapping.head(5)

,Título,Precio,Dirección,Enlace,Tipo moneda
0,Spot Nueva Kennedy,19,"Av. Manquehue Nte. 958, Las Condes, Parque Ara...",https://portalinmobiliario.com/MLC-1555797337-...,UF
1,Augusto Leguía,815000,"Augusto Leguía Nte. 70, Barrio El Golf, Las Co...",https://portalinmobiliario.com/MLC-2648931672-...,CLP
2,Vespucio Switch,879178,"Av. Américo Vespucio Sur 345, Metro Escuela Mi...",https://portalinmobiliario.com/MLC-1525161537-...,CLP
3,Somma Asturias - Edificio Multifamily,830000,"Asturias 77, Metro Escuela Militar, Las Condes...",https://portalinmobiliario.com/MLC-1408634227-...,CLP
4,Dpto Amoblado Remodelado 2d+2b Los Militares C...,27,"Los Militares 4717, Metro Escuela Militar, Las...",https://portalinmobiliario.com/MLC-2814352114-...,UF


In [11]:
len(data_scrapping)

96

In [12]:
data_scrapping['Precio'].describe()

count    9.600000e+01
mean     4.866648e+05
std      4.575840e+05
min      1.600000e+01
25%      2.500000e+01
50%      6.500000e+05
75%      8.700000e+05
max      1.300000e+06
Name: Precio, dtype: float64

### Valor UF

In [14]:
# Configuramos el driver para el navegador Edge
driver = webdriver.Edge()
driver.set_window_size(1200, 800)  # Ancho: 1200px, Alto: 800px

driver.get(URL_UF)

# Establecemos una espera implícita de 5 segundos para que la página cargue completamente
driver.implicitly_wait(10)
valor_uf = driver.find_element(By.XPATH, "//span[@class='vpr']").text
valor_uf = float(valor_uf.replace('$', '').replace(' ', '').replace('.', '').replace(',', '.'))
print(f'El valor de la UF actualizado es: {valor_uf}')

time.sleep(5)
driver.close() 


El valor de la UF actualizado es: 38411.74


In [15]:
data_scrapping_procesada = data_scrapping.copy()
data_scrapping_procesada['Precio'] = data_scrapping_procesada.apply(lambda row: row['Precio']*valor_uf \
    if row['Tipo moneda'] == 'UF' else row['Precio'], axis = 1)
data_scrapping_procesada['Precio'] = data_scrapping_procesada['Precio'].astype(int)
data_scrapping_procesada.drop(['Tipo moneda'], axis = 1, inplace = True)
data_scrapping_procesada = data_scrapping_procesada[(data_scrapping_procesada['Precio']>=MONTO_MINIMO) & (data_scrapping_procesada['Precio']<=MONTO_MAXIMO)]
data_scrapping_procesada.head(5)

,Título,Precio,Dirección,Enlace
0,Spot Nueva Kennedy,729823,"Av. Manquehue Nte. 958, Las Condes, Parque Ara...",https://portalinmobiliario.com/MLC-1555797337-...
1,Augusto Leguía,815000,"Augusto Leguía Nte. 70, Barrio El Golf, Las Co...",https://portalinmobiliario.com/MLC-2648931672-...
2,Vespucio Switch,879178,"Av. Américo Vespucio Sur 345, Metro Escuela Mi...",https://portalinmobiliario.com/MLC-1525161537-...
3,Somma Asturias - Edificio Multifamily,830000,"Asturias 77, Metro Escuela Militar, Las Condes...",https://portalinmobiliario.com/MLC-1408634227-...
4,Dpto Amoblado Remodelado 2d+2b Los Militares C...,1037116,"Los Militares 4717, Metro Escuela Militar, Las...",https://portalinmobiliario.com/MLC-2814352114-...


In [16]:
data_scrapping_procesada['Precio'].describe()

count    9.600000e+01
mean     8.891772e+05
std      2.016921e+05
min      5.500000e+05
25%      7.162500e+05
50%      8.745890e+05
75%      1.075528e+06
max      1.300000e+06
Name: Precio, dtype: float64

### Procesamiento direcciones

Direcciones con problemas: intervalos numéricos sin dar la dirección exacta

Se consulta chat GPT por el pattern exacto a ocupar

In [17]:
# Expresión regular para buscar direcciones con un intervalo numérico
pattern = r'\b(\d+(?:\.\d+)?(?:e[+-]?\d+)?)\s*-\s*(\d+(?:\.\d+)?(?:e[+-]?\d+)?)\b'
data_scrapping_procesada[data_scrapping_procesada['Dirección'].str.contains(pattern, flags=re.IGNORECASE, regex=True)].head(10)

C:\Users\Diego\AppData\Local\Temp\ipykernel_16152\2147432578.py:5: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  data_scrapping_procesada[data_scrapping_procesada['Dirección'].str.contains(pattern, flags=re.IGNORECASE, regex=True)].head(10)


,Título,Precio,Dirección,Enlace
12,"Guanabara 2d2b + Est Y Bdga, Las Condes",1180000,"Guanabara 900 - 1200, Las Condes, Rotonda Aten...",https://portalinmobiliario.com/MLC-2814805660-...
18,Arrienda Studio Nuevo Amoblado Moderno Edifici...,690000,"Las Tranqueras 1 - 300, Las Condes, Alto Las C...",https://portalinmobiliario.com/MLC-2812221716-...
29,Metro Manquehue/4 Amplios Dormitorios/la Recon...,1050000,"La Reconquista 863, 7.5801e+06 - 7.5804e+06 La...",https://portalinmobiliario.com/MLC-2809126410-...
30,Metro H De Magallanes/nuevo/sin Uso/1 Dormitor...,729823,"Fontana Rosa 7106, 7.5705e+06 - 7.5708e+06 Las...",https://portalinmobiliario.com/MLC-1558905789-...
32,Amoblado/metro H.de Magallanes/1dormitorio/est...,768234,"Los Ilanes 140, 7.5603e+06 - 7.5606e+06 Las Co...",https://portalinmobiliario.com/MLC-2808819448-...
33,El Golf/2 Dormitorios/2 Baños/estacionamiento,870000,"Roger De Flor 2700 - 3000, Las Condes, Barrio ...",https://portalinmobiliario.com/MLC-2808291140-...
35,"Oportunidad, Departamento De 1 Dormitorio Con ...",700000,"Hnos Cabot 7800 - 8100, Las Condes, Alto Las C...",https://portalinmobiliario.com/MLC-2814626488-...
39,"Excelente Dep 3h2b, 1 Est, Piscina A Pasos Met...",1075528,"Escandinavia 110, 7.5606e+06 - 7.5609e+06 Las ...",https://portalinmobiliario.com/MLC-2806717968-...
44,Excelente Depto. Un Dorm. Amoblado A Pasos Met...,730000,"Evaristo Lillo 29, Depto 1 - 300, Las Condes, ...",https://portalinmobiliario.com/MLC-2805523956-...
45,Excelente Depto. 1 Dorm. Amoblado Metro Alcantara,940000,"Alsacia 1 - 300, Las Condes, Barrio El Golf, L...",https://portalinmobiliario.com/MLC-1560527075-...


In [18]:
def corregir_direccion(direccion):
    match = re.search(pattern, direccion)
    if match:
        inicio, fin = map(float, match.groups())
        promedio = (inicio + fin) / 2
        return re.sub(pattern, f"{promedio:.0f}", direccion)
    else:
        return direccion


# Crear nueva dirección procesada
data_scrapping_procesada['Dirección'] = data_scrapping_procesada['Dirección'].apply(corregir_direccion)
data_scrapping_procesada[data_scrapping_procesada['Dirección'].str.contains(pattern, flags=re.IGNORECASE, regex=True)]



C:\Users\Diego\AppData\Local\Temp\ipykernel_16152\2078627988.py:13: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  data_scrapping_procesada[data_scrapping_procesada['Dirección'].str.contains(pattern, flags=re.IGNORECASE, regex=True)]


,Título,Precio,Dirección,Enlace


In [19]:
data_scrapping_procesada.head(5)

,Título,Precio,Dirección,Enlace
0,Spot Nueva Kennedy,729823,"Av. Manquehue Nte. 958, Las Condes, Parque Ara...",https://portalinmobiliario.com/MLC-1555797337-...
1,Augusto Leguía,815000,"Augusto Leguía Nte. 70, Barrio El Golf, Las Co...",https://portalinmobiliario.com/MLC-2648931672-...
2,Vespucio Switch,879178,"Av. Américo Vespucio Sur 345, Metro Escuela Mi...",https://portalinmobiliario.com/MLC-1525161537-...
3,Somma Asturias - Edificio Multifamily,830000,"Asturias 77, Metro Escuela Militar, Las Condes...",https://portalinmobiliario.com/MLC-1408634227-...
4,Dpto Amoblado Remodelado 2d+2b Los Militares C...,1037116,"Los Militares 4717, Metro Escuela Militar, Las...",https://portalinmobiliario.com/MLC-2814352114-...


## Geocoding y places

In [ ]:

# Función para obtener datos de geocoding
def obtener_datos_geocoding(direccion):
    url = f"https://maps.googleapis.com/maps/api/geocode/json?address={direccion}&key={API_KEY}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data["results"]:
            geometry = data["results"][0]["geometry"]
            location = geometry["location"]
            place_id = data["results"][0]["place_id"]
            return {'geometry': {'location': location}, 'place_id': place_id}
    return {'geometry': {'location': {'lat': None, 'lng': None}}, 'place_id': None}

# Añadir datos al DataFrame
data_scrapping_procesada['Geocoding_Data'] = data_scrapping_procesada['Dirección'].apply(obtener_datos_geocoding)

# Separar columnas para latitud, longitud y place_id
data_scrapping_procesada['Latitud'] = data_scrapping_procesada['Geocoding_Data'].apply(lambda x: x['geometry']['location']['lat'])
data_scrapping_procesada['Longitud'] = data_scrapping_procesada['Geocoding_Data'].apply(lambda x: x['geometry']['location']['lng'])
data_scrapping_procesada['Place_ID'] = data_scrapping_procesada['Geocoding_Data'].apply(lambda x: x['place_id'])

# Mostrar el DataFrame actualizado
print(data_scrapping_procesada[['Dirección', 'Latitud', 'Longitud', 'Place_ID']])